### Data aggregation

In [ ]:
!pip install --upgrade cerebras_cloud_sdk -q

In [ ]:
import os
from tqdm.notebook import tqdm, trange
import json
import re
from cerebras.cloud.sdk import Cerebras

dataset = load_dataset("openai/gsm8k", 'socratic', split="train")
questions = [dataset[i]['question'] for i in trange(len(dataset))]

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

n = len(questions)
output_path = "dataset_ru1_updated.json"

client = Cerebras(
    api_key=user_secrets.get_secret("data_apis"),
)

for i in trange(n):
    response = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": """Вы — опытный переводчик и решатель задач.
Переведите текст на русский, сохранив математические формулы и структуру без изменений.
После этого выведи последовательные вопросы и ответы на них, позволяющие решить задачу.
Формат вывода: 'Q: Условие задачи на русском языке \n SQ1: первый подвопрос \n A1: ответ на первый подвопрос\n ... A: ответ на задачу'.
Например, 'Q:У Кати было 3 кружки, у Кости 5 кружек. Сколько суммарно у них было кружек? \n SQ1: Сколько будет 3 плюс 5? \n A1: 3+5=8, значит будет 8 \n A: Ответ: будет 8 кружек. \n '.\n""" + questions[i],
                    }
            ],
                model="qwen-3-235b-a22b",
            )
    answer = response.choices[0].message.content.strip()
    # answer = answer.split("</think>")[1].lsplit().split()
    with open(output_path, "a", encoding="utf-8") as f:
        f.write(json.dumps({"answer": answer.split("</think>")[1]}, ensure_ascii=False) + "\n")

In [ ]:
pattern = r'(SQ\d+:\s.*?)(?=\n(?:SQ\d+|$))'

def parse_qa(text):
    text = text.strip()
    # print(text)
    if "Q:" in text:
        _, rest = text.split("Q:", 1)
    else:
        return None, [], ""
        
    if "A:" in rest:
        question_part, final_answer = rest.split("A:", 1)
    else:
        return None, [], ""
        
    if "SQ" in question_part:
        question_part, answer_part = question_part.split("SQ", 1)
        answer_part = "SQ" + answer_part
    else:
        return None, [], ""
    steps = re.split(r'(?:SQ\d+:\s|A:\s)', answer_part.strip())
    keys = re.findall(r'(SQ\d+:\s.*?)(?=(?:SQ|$))', answer_part, re.DOTALL)
    
    reasoning_steps = []
    for key, step in zip(keys, steps[1:]):
        match = re.search(r'SQ\d+:\s*(.*?)\s*\n\s*A\d+:\s*(.*)', key, re.DOTALL)

        if match:
            question = match.group(1).strip()
            answer = match.group(2).strip()
            reasoning_steps.append({
                "step": question,
                "answer": answer
            })
    # final_answer = steps[-1].strip()

    return question_part.strip(), reasoning_steps, final_answer.strip()

text = "\n\nQ: Наталья продала 48 заколок своим подругам в апреле, а в мае она продала в два раза меньше. Сколько заколок Наталья продала всего в апреле и мае?  \n SQ1: Как вычислить количество заколок, проданных в мае?  \n A1: Количество проданных в мае заколок равно 48 ÷ 2 = 24.  \n SQ2: Как найти общее количество заколок, проданных в апреле и мае?  \n A2: Нужно сложить количество за апрель и май: 48 + 24 = 72.  \n A: Ответ: Наталья продала всего 72 заколки."
parse_qa(text)

In [ ]:
input_files = ["/kaggle/input/ru-gsm8k/dataset_ru1_updated(5).json",
               "/kaggle/input/ru-gsm8k/dataset_ru1_updated(7).json",
               "/kaggle/input/ru-gsm8k/dataset_ru1_updated(8).json"]
output_file = "structured_cot_dataset.jsonl"

for file in input_files:
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line.strip())
            full_text = item["answer"]
            question, steps, answer = parse_qa(full_text)
            if question and answer:
                entry = {
                    "prompt": question,
                    "reasoning_steps": steps,
                    "final_answer": answer
                }
                with open(output_file, "w", encoding="utf-8") as out_f:
                    out_f.write(json.dumps(entry, ensure_ascii=False) + "\n")
print(f"Структурированный датасет сохранён в {output_file}")

### QA model education

In [ ]:
!pip install \
flash-attn==2.7.4.post1 \
scipy==1.15.2 \
torch==2.6.0 \
cffi==1.17.1 \
transformers==4.53.2 \
peft==0.14.0 \
accelerate==1.5.1 \
trl==0.19.1 \
bitsandbytes==0.45.3 \
datasets==3.3.2 \
huggingface-hub==0.33.4 \
safetensors==0.5.3 \
pandas==2.2.3 \
matplotlib==3.10.1 \
numpy==1.26.4 \
wandb
-q

In [ ]:
import os
import torch
from datasets import load_dataset, Dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
from tqdm.notebook import tqdm, trange
import wandb

In [ ]:
wandb.login(key=user_secrets.get_secret("wandb_api"))

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device used: {device}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-1.7B"
)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-1.7B",
    attn_implementation="sdpa",
    torch_dtype=torch.bfloat16,
    device_map=device
)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/kaggle/input/ru-gsm8k-strustered/structured_dataset_train.jsonl", split="train")

def generate_progressive_examples(example):
    prompt = example["prompt"]
    steps = example.get("reasoning_steps", [])
    final_answer = example.get("final_answer", "")

    examples = []
    context = f"{prompt}\n"

    for i in range(len(steps)):
        step = steps[i]
        context += f"{step['step']}\n"
        
        examples.append({
            "messages": [{'content': 'Ты полезный ассистент. Если в запросе последнее предложение вопрос - надо ответить на него. Иначе надо дать итоговый ответ по задаче, основываясь на введенном тексте.','role': 'system'},
                         {'content': context,'role': 'user'},
                         {'content': f"{step['answer']}\n",'role': 'assistant'}]
        })
        context += f"{step['answer']}\n"
    
    if final_answer:
        examples.append({
            "messages": [{'content': 'Ты полезный ассистент. Если в запросе последнее предложение вопрос - надо ответить на него. Иначе надо дать итоговый ответ по задаче, основываясь на введенном тексте.','role': 'system'},
                         {'content': context,'role': 'user'},
                         {'content': f"{final_answer}\n",'role': 'assistant'}]
        })

    return {"progressive_examples": examples}

dataset_expanded = dataset.map(
    generate_progressive_examples,
    remove_columns=dataset.column_names,
    batched=False
)

from itertools import chain

def flatten_dataset(batch):
    return {
        "messages": list(chain.from_iterable(batch["progressive_examples"]))
    }

dataset_flat = dataset_expanded.map(
    flatten_dataset,
    batched=True,
    remove_columns=["progressive_examples"]
)
df_train = dataset_flat.map(lambda x: x["messages"])

In [ ]:
def convert_example(example):
    messages = example["messages"]

    prompt_messages = messages[:-1]
    assistant = messages[-1]["content"]

    prompt = tokenizer.apply_chat_template(
        prompt_messages,
        add_generation_prompt=True,
        tokenize=False
    )

    return {"prompt": prompt, "completion": assistant}

def tokenize_and_align_labels(example):
    prompt = example["prompt"]
    completion = example["completion"]

    full_text = prompt + completion
    inputs = tokenizer(full_text, truncation=True, max_length=1024)

    prompt_len = len(tokenizer(prompt, add_special_tokens=False)["input_ids"])
    labels = [-100] * prompt_len + inputs["input_ids"][prompt_len:]

    inputs["labels"] = labels
    return inputs

df_train = df_train.map(convert_example, remove_columns=df_train.column_names)


### LoRA fine-tuning

In [ ]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    # the rank of the adapter, the lower the fewer parameters you'll need to train
    r=8,
    lora_alpha=16, # multiplier, usually 2*r
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
)
model = get_peft_model(model, config)
model

In [ ]:
model.print_trainable_parameters()

In [ ]:
sft_config = SFTConfig(
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': False},
    gradient_accumulation_steps=8,
    per_device_train_batch_size=4,
    # auto_find_batch_size=True,

    max_seq_length=1024,
    packing=False,
    completion_only_loss=True,
    # assistant_only_loss=True,

    num_train_epochs=2,
    learning_rate=3e-4,
    optim='paged_adamw_8bit',

    logging_steps=10,
    logging_dir='./logs',
    output_dir='./qwen_finetuned_lora',
    report_to=["wandb"],

    save_strategy='epoch',
    save_total_limit=2,
    bf16=True
)

In [ ]:
shuffled_train = df_train.shuffle()
df_train_part = shuffled_train.select(range(20000))

In [ ]:
trainer_lora = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=df_train_part,
)

In [ ]:
trainer_lora.train()

In [ ]:
!zip -r /kaggle/working/results.zip /kaggle/working/qwen_finetuned_lora

### Two models

In [ ]:
class TwoStageModel:
    def __init__(self, model_qg_path, model_qa_path):
        self.model_qg = AutoModelForCausalLM.from_pretrained(
            model_qg_path,
            # attn_implementation="sdpa",
            torch_dtype=torch.bfloat16,
            device_map=device
        )
        self.model_qa = AutoModelForCausalLM.from_pretrained(
            model_qa_path,
            # attn_implementation="sdpa",
            torch_dtype=torch.bfloat16,
            device_map=device
        )
        
        self.tokenizer_qg = AutoTokenizer.from_pretrained(model_qg_path)
        self.tokenizer_qa = AutoTokenizer.from_pretrained(model_qa_path)
        
        self.model_qg.eval()
        self.model_qa.eval()

        self.QG_PROMPT = "Ты полезный ассистент. Тебе вводят задачу и существующие этапы решения. Выведи следующий вопрос, помогающий решить задачу."
        self.QA_PROMPT = "Ты полезный ассистент. Если в запросе последнее предложение вопрос - надо ответить на него. Иначе надо дать итоговый ответ по задаче, основываясь на введенном тексте, в формате: 'Ответ: *твой ответ*'."
    
    def gen_prompt(self, tokenizer, sentence, system_prompt):
        converted_sample = [
            {'content': system_prompt, 'role': 'system'},
            {'content': sentence, 'role': 'user'}
        ]
        prompt = tokenizer.apply_chat_template(
            converted_sample, tokenize=False, add_generation_prompt=True
        )
        return prompt
        
    def generate(self, prompt, max_new_tokens=100, **kwargs):
        for _ in range(10):
            inputs1 = self.tokenizer_qg(self.gen_prompt(self.tokenizer_qg, prompt, self.QG_PROMPT),
                                        return_tensors="pt").to(self.model_qg.device)
            
            with torch.no_grad():
                output1 = self.model_qg.generate(
                    **inputs1,
                    max_new_tokens=max_new_tokens,
                    **kwargs
                )
            
            intermediate_text = self.tokenizer_qg.decode(output1[0, inputs1['input_ids'].shape[1]:],
                                                         skip_special_tokens=True)
            intermediate_text = intermediate_text.replace("<think>", "")
            intermediate_text = intermediate_text.replace("</think>", "")
            
            prompt += intermediate_text
            inputs2 = self.tokenizer_qa(self.gen_prompt(self.tokenizer_qa, prompt, self.QA_PROMPT),
                                        return_tensors="pt").to(self.model_qa.device)
            
            with torch.no_grad():
                output2 = self.model_qa.generate(
                    **inputs2,
                    max_new_tokens=max_new_tokens,
                    **kwargs
                )
            output = self.tokenizer_qa.decode(output2[0, inputs2['input_ids'].shape[1]:],
                                         skip_special_tokens=True)
            if "Ответ:" in output:
                output = output.replace("Ответ:", "")
                return "<think>" + prompt + "</think>" + output
            prompt += output
        return output

In [ ]:
cascade_model = TwoStageModel("AccessAndrei/qwen3-1.7b-merged", "annodomini704/qwen3-1.7B_SoT_QA")

In [ ]:
cascade_model.generate("У Кати было 2 яблока, у Пети было 3 яблока, а у Коли 4 яблока. Сколько всего было яблок у детей?")

### Evaluate on MERA datasets

In [1]:
def cut_answer(line):
    if "</think>" not in line:
        return line
    return  re.sub(r'[^\w\s]', '', line.split("</think>")[1]).strip()

def count_acc(model, dataset):
    good_predicts = 0
    for i in trange(len(dataset)):
        if cut_answer(model.generate(dataset[i]["input"])).strip() == dataset[i]["output"]:
            good_predicts += 1
        if i%10==0:
            print(i, good_predicts / (i+1))
    
    return good_predicts / len(dataset)

In [ ]:
lcs = datasets.load_dataset("MERA-evaluation/MERA", "lcs", split="public_test")
def make_input(el):
    input = el["instruction"].replace("{inputs}", el["inputs"])
    return {"input": input, "output": el["outputs"]}

lcs_prepared = lcs.map(make_input, remove_columns=lcs.column_names, batched=False)
count_acc(cascade_model, lcs_prepared)

In [ ]:
rwsd = datasets.load_dataset("MERA-evaluation/MERA", "rwsd", split="train")
def make_input(el):
    input = el["instruction"].replace("{text}", el["inputs"]["text"])
    input = input.replace("{span1_text}", el["inputs"]["span1_text"])
    input = input.replace("{span2_text}", el["inputs"]["span2_text"])
    return {"input": input, "output": el["outputs"]}

rwsd_prepared = rwsd.map(make_input, remove_columns=rwsd.column_names, batched=False)
count_acc(cascade_model, rwsd_prepared)